In [6]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_14.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub = "14_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub, max_tau)

filename_cross = "14_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross)


Submatrix 14_sub_mat_1.csv has been saved.
submatrix 14_cross_matrix_2_3.csv has been saved


38099.0   38101.0   38103.0   38105.0   39001.0   39003.0   39005.0  \
21083.0  0.075235   0.41835  0.193152  0.096404 -0.355258  0.238233  0.450321   
21085.0 -0.086755 -0.430519 -0.319431  0.181884 -0.380902 -0.239004 -0.484568   
21087.0  0.135765 -0.318329 -0.159556  0.150624 -0.377459  0.285787  0.515655   
21089.0  0.145483 -0.436618  0.206936  0.155148  0.357053  0.265391  0.425933   
21091.0  0.253044   0.42256  0.225667  0.355339 -0.394058  0.321625  0.409053   
...           ...       ...       ...       ...       ...       ...       ...   
38089.0 -0.509668  0.552323  0.359501 -0.513799 -0.400996 -0.621266 -0.385723   
38091.0 -0.130746 -0.260228 -0.205472 -0.329359  -0.09633  0.156452 -0.293409   
38093.0  0.603597  0.454847  0.492556  0.627481  0.420289  0.570656  0.363897   
38095.0 -0.396086 -0.297582  0.480224  0.380787 -0.367815 -0.368068 -0.204467   
38097.0 -0.473219  0.290493  0.177815  0.364803  0.280012  0.358086 -0.326301   

          39007.0   39009.0   39011.0  ...   56027.0   56029.0   56031.0  \
21083.0  0.072466  0.110093  0.229873  ... -0.518339  0.566096  0.477483   
21085.0  0.112786  0.127661 -0.325037  ... -0.393973  0.463743  0.389365   
21087.0  0.086972  0.151941 -0.277668  ... -0.472417  0.487195 -0.455903   
21089.0  0.160843  0.143997   0.24323  ... -0.454383  0.506805  0.522747   
21091.0  0.245462  0.285566   0.33078  ...  0.425556   0.42211 -0.399198   
...           ...       ...       ...  ...       ...       ...       ...   
38089.0  0.520095 -0.541889 -0.568376  ...  0.271134  0.381024 -0.108848   
38091.0  0.255302  0.169323  0.145982  ... -0.237434 -0.247209  -0.21037   
38093.0  0.572211  0.596805  0.592731  ...  0.264862  0.283744 -0.102018   
38095.0  0.397635  -0.44937  -0.34137  ... -0.289622 -0.271523         0   
38097.0  0.371036  0.474387  0.365371  ... -0.328917 -0.255033 -0.192294   

          56033.0   56035.0   56037.0   56039.0   56041.0   56043.0   56045.0  
21083.0  0.758194 -0.420886  0.335489   0.73511  0.555295  0.307524   0.60741  
21085.0  0.643847 -0.412429 -0.266466  0.663266  0.508317  0.377183 -0.571574  
21087.0  0.528651 -0.402202 -0.171146   0.44726  0.526117 -0.359775 -0.441396  
21089.0  0.669178 -0.466119 -0.338673  0.732997   0.52461  0.321546   0.58744  
21091.0  0.561012 -0.440926  0.212167  0.558789  0.554727  0.281115  0.502142  
...           ...       ...       ...       ...       ...       ...       ...  
38089.0  0.374437 -0.369456  0.349978 -0.327213 -0.304676 -0.315335 -0.316321  
38091.0 -0.271242 -0.100453 -0.426901 -0.157978 -0.147231 -0.210504 -0.181639  
38093.0  0.253759 -0.340247  0.376269 -0.372377  0.262375  0.241243 -0.216797  
38095.0 -0.156451 -0.211866 -0.172952  -0.08928 -0.176056  0.204223         0  
38097.0 -0.414938 -0.260254 -0.238445 -0.386629  0.263204  0.251123 -0.254815  

[1000 rows x 1104 columns]